In [1]:
#| default_exp clean
from hn_jobs_chat.vars import postsTableName
from hn_jobs_chat.db import connectToDB

key = 'job_requirements'
tableName = postsTableName + "_" + key


In [9]:
conn, cursor = connectToDB()

key = 'job_requirements'
tableName = postsTableName + "_" + key

query = """ALTER TABLE """+tableName+""" ADD COLUMN IF NOT EXISTS embedding float[1536]"""
cursor.execute(query)
conn.commit()

conn.close()


In [ ]:
from openai import OpenAI
client = OpenAI()

conn, cursor = connectToDB()

key = 'techstackitems'
tableName = postsTableName + "_" + key

query = """ALTER TABLE """+tableName+""" ADD COLUMN IF NOT EXISTS embedding float[1536]"""
cursor.execute(query)
conn.commit()

conn.close()



In [37]:
from openai import OpenAI
client = OpenAI()

conn, cursor = connectToDB()

key = 'techstackitems'
tableName = postsTableName + "_" + key

query = """SELECT * FROM """ + tableName

cursor.execute(query)

data = cursor.fetchall()

for datum in data:
    item_id, text = datum[0],datum[2]

    search_query = """SELECT * FROM """ + tableName + """ WHERE tech = %s AND embedding IS NOT NULL"""
    cursor.execute(search_query, (text,))
    search_results = cursor.fetchone()

    if search_results is not None:
        embedding = search_results[3]
    else:
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )

        embedding = (response.data[0].embedding)

    query = """UPDATE """ + tableName + """ SET embedding = %s WHERE id = %s"""
    cursor.execute(query, (embedding, item_id))
    conn.commit()


conn.close()

In [42]:
from openai import OpenAI
client = OpenAI()

conn, cursor = connectToDB()

tableSuffix = 'job_requirements'
tableName = postsTableName + "_" + tableSuffix
key = 'job_requirements'
query = """SELECT * FROM """ + tableName

cursor.execute(query)

data = cursor.fetchall()

for datum in data:
    # if an identical item has already been processed, just use that embedding
    # otherwise, retreive and embedding

    item_id, text = datum[0],datum[2]

    print('processing: ' + str(item_id))

    search_query = """SELECT * FROM """ + tableName + """ WHERE """+key+""" = %s AND embedding IS NOT NULL"""
    cursor.execute(search_query, (text,))
    search_results = cursor.fetchone()

    if search_results is not None:
        embedding = search_results[3]
    else:
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )

        embedding = (response.data[0].embedding)

    query = """UPDATE """ + tableName + """ SET embedding = %s WHERE id = %s"""
    cursor.execute(query, (embedding, item_id))
    conn.commit()

conn.close()

processing1
processing2
processing3
processing4
processing5
processing6
processing7
processing8
processing9
processing10
processing11
processing12
processing13
processing14
processing15
processing16
processing17
processing18
processing19
processing20
processing21
processing22
processing23
processing24
processing25
processing26
processing27
processing28
processing29
processing30
processing31
processing32
processing33
processing34
processing35
processing36
processing37
processing38
processing39
processing40
processing41
processing42
processing43
processing44
processing45
processing46
processing47
processing48
processing49
processing50
processing51
processing52
processing53
processing54
processing55
processing56
processing57
processing58
processing59
processing60
processing61
processing62
processing63
processing64
processing65
processing66
processing67
processing68
processing69
processing70
processing71
processing72
processing73
processing74
processing75
processing76
processing77
processi

The rest of the owl.

In [ ]:
from openai import OpenAI
client = OpenAI()

conn, cursor = connectToDB()

tableSuffix = ''
tableName = postsTableName + "_" + tableSuffix
key = 'job_requirements'
query = """SELECT * FROM """ + tableName

cursor.execute(query)

data = cursor.fetchall()

for datum in data:
    # if an identical item has already been processed, just use that embedding
    # otherwise, retreive and embedding

    item_id, text = datum[0],datum[2]

    print('processing: ' + str(item_id))

    search_query = """SELECT * FROM """ + tableName + """ WHERE """+key+""" = %s AND embedding IS NOT NULL"""
    cursor.execute(search_query, (text,))
    search_results = cursor.fetchone()

    if search_results is not None:
        embedding = search_results[3]
    else:
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )

        embedding = (response.data[0].embedding)

    query = """UPDATE """ + tableName + """ SET embedding = %s WHERE id = %s"""
    cursor.execute(query, (embedding, item_id))
    conn.commit()

conn.close()



In [7]:
len(response.data[0].embedding)

1536